In [ ]:
!pip install pathway --upgrade --quiet  # This cell may take a few seconds to execute.

In [ ]:
#importing important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import math

from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')




In [ ]:
!git clone https://github.com/Abhishek690012/Capstone_mini_project.git
df = pd.read_csv("Capstone_mini_project/dataset.csv")
df.head()

In [ ]:
#converting the year from  two digit form to four digit form
df['Year'] = df['LastUpdatedDate'].str[6:8]
df['LastUpdatedDate'] = df['LastUpdatedDate'].str[:-2]
df['Year'] = '20'+df['Year']
df['LastUpdatedDate' ]= df['LastUpdatedDate']+df['Year']
df['LastUpdatedTime'] = df['LastUpdatedTime'] + ':00'
df.drop(columns = ['Year'],inplace = True)
df.head()

In [ ]:
#Encoding the nearby traffic condition column
conditions = [
    df['TrafficConditionNearby'] == 'low',
    df['TrafficConditionNearby'] == 'average',
    df['TrafficConditionNearby'] == 'high'
]
choices = [1, 2, 3]  # Assign any values you want for each category

df['TrafficCondition'] = np.select(conditions, choices)
df.head()

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%m-%d-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)
df.drop(columns = ['Latitude','Longitude','VehicleType','TrafficConditionNearby','IsSpecialDay'],inplace = True)

In [ ]:
def pseudo_price(cap,occ,traffic_cond,queue_len):
  ini_coeff = [0.6,1.5,1.1,2.3,1]
  demand = -cap*ini_coeff[0] + occ *ini_coeff[1] - traffic_cond*ini_coeff[2]+ queue_len*ini_coeff[3]
  #sigmoid function for normalization
  import math

  x = 8
  if demand > 0:
      normalized_demand = 1.0/(1.0+math.exp(-demand))
      price = 10.0 + x*normalized_demand
      return price
  else:
      price = 10.0
      return price

In [ ]:
#Defining the schema for the pipeline
class schema(pw.Schema):
  Timestamp: str
  Occupancy: int
  Capacity: int
  SystemCodeNumber : str
  TrafficCondition : int
  QueueLength:int
  # IsSpecialDay : int

df.to_csv("1.csv", index=False)


In [ ]:
curr = pd.read_csv('/content/1.csv')
complete_input = curr.copy()

curr_in = pw.demo.replay_csv("/content/1.csv", schema=schema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"

curr_input = curr_in.with_columns(
    LastUpdatedDate = curr_in.Timestamp.dt.strptime(fmt),
    day = curr_in.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

#defining the window  behaviour
window = (
    curr_input.windowby(
        pw.this.LastUpdatedDate,
        window = pw.temporal.tumbling(datetime.timedelta(days = 1)),
        instance = pw.this.SystemCodeNumber,
        behavior = pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ=pw.reducers.avg(pw.this.Occupancy)/pw.reducers.max(pw.this.Occupancy), #Max Occupancy during the day
        #occ_min=pw.reducers.min(pw.this.Occupancy), #Min Occupancy during the day
        cap=pw.reducers.avg(pw.this.Capacity)/pw.reducers.max(pw.this.Capacity),  #Capacity
        traffic_cond = pw.reducers.avg(pw.this.TrafficCondition)/pw.reducers.max(pw.this.TrafficCondition),
        queue_len = pw.reducers.avg(pw.this.QueueLength)/pw.reducers.max(pw.this.QueueLength),
        Code = pw.reducers.earliest(pw.this.SystemCodeNumber)
    )
    .with_columns(
        SystemCodeNumber = pw.this.Code,
        price = pw.apply(
            pseudo_price,
            pw.this.cap,
            pw.this.occ,
            pw.this.traffic_cond,
            pw.this.queue_len,
        )

      )
)

In [ ]:
#upper limit or the index for plotting is 13 i.e. you can plot for 14 different parking lots
unique_codes = df['SystemCodeNumber'].unique()


# Activate the Panel extension to enable interactive visualizations
pn.extension()


# Creating the window in which the plots will be stored.
def price_plotter_Labeled(name):
    def price_plotter(source):
        from bokeh.plotting import figure
        #fig = figure(title=f"Price Plot for {name}")
        fig = bokeh.plotting.figure(
          height=300,
          width=600,
          title=f"Price Plot for {name}",
          x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
        )

        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter(x="t", y="price", source=source, size=4, color="red", marker="circle")
        return fig
    return price_plotter

#upper limit or the index for plotting is 13 i.e. you can plot for 14 different parking lots
unique_codes = df['SystemCodeNumber'].unique()

plots = []#for storing the plots

for i in range(unique_codes.size):
  filtered = window.filter(window.SystemCodeNumber == unique_codes[i])
  viz = filtered.plot(price_plotter_Labeled(unique_codes[i]), sorting_col="t")
  plots.append(viz)


dashboard = pn.Column(*plots)
dashboard.servable()

In [ ]:
pw.run() #Run this line after running the whole file with run all option.